Reference: https://colab.research.google.com/drive/1HEcOKFwK-Bk8T3R_IIqBhNjPO5opmCO0#scrollTo=CK6d-kNzO2hX  
Optimizer Introduction: https://www.comet.ml/docs/python-sdk/introduction-optimizer/

In [1]:
from comet_ml import Experiment
import keras
from keras.datasets import mnist
from keras.layers import Dense,Dropout
from keras.models import Sequential, Model
from keras.optimizers import  RMSprop
from keras.callbacks import Callback
from keras.preprocessing import image

C:\Anaconda\envs\py36gpu\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
(x_train, y_train),(x_test,y_test)=mnist.load_data()

In [3]:
x_train.shape

(60000, 28, 28)

In [4]:
x_train=x_train.reshape(60000,784)
x_test=x_test.reshape(10000,784)
x_train = x_train.astype("float32")
x_test= x_test.astype("float32")
x_train /=255
x_test /=255

print(x_train.shape[0],"train samples")
print(x_test.shape[0],"test samples")


60000 train samples
10000 test samples


In [5]:
y_train.shape

(60000,)

In [6]:
y_train=keras.utils.to_categorical(y_train,10)
y_test=keras.utils.to_categorical(y_test,10)
y_train[0]

array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0.], dtype=float32)

In [7]:
def build_model(input_shape=(784,)):
    model=Sequential()
    model.add(Dense(128,activation="sigmoid",input_shape=(784,)))
    model.add(Dense(256,activation="sigmoid"))
    model.add(Dense(128,activation="sigmoid"))
    model.add(Dense(10,activation="softmax"))
    model.compile(
        loss="categorical_crossentropy",
        optimizer=RMSprop(),
        metrics=["accuracy"]
    )
    return model


        

In [8]:
model = build_model()
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 128)               100480    
_________________________________________________________________
dense_2 (Dense)              (None, 256)               33024     
_________________________________________________________________
dense_3 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_4 (Dense)              (None, 10)                1290      
Total params: 167,690
Trainable params: 167,690
Non-trainable params: 0
_________________________________________________________________


In [9]:
import logging
def train(x_train,y_train,x_test,y_test,batch,epoch):
    model=build_model()
    model.fit(
        x_train,
        y_train,
        batch_size=batch,
        epochs=epoch,
        validation_data=(x_test,y_test)
    )
    score=model.evaluate(x_test,y_test)
    logging.info("score %s",score)


**Create an  Experiment:**

In [11]:
config={
    "algorithm" : "bayes",
    "parameters": {
        "batch":{"type":"integer", "min":60 ,"max":100},
        "epoch":{"type":"integer", "min":5,"max":20}
    }
    ,
    "spec":{
        "metric":"zzxx",
        "objective":"maximize",
        "retryAssignLimit":1
    }
}

from comet_ml import Optimizer 
opt=Optimizer(config,api_key="6krXLYdn4mMFKPsF8jwrFwXtu",project_name="optimizer-search-02")

COMET WARNING: Passing Experiment through Optimizer constructor is deprecated; pass them to Optimizer.get_experiments or Optimizer.next
COMET INFO: COMET_OPTIMIZER_ID=f71cdd9b4b874da2ba7a186c31c380b2
COMET INFO: Using optimizer config: {'algorithm': 'bayes', 'configSpaceSize': 'infinite', 'endTime': None, 'id': 'f71cdd9b4b874da2ba7a186c31c380b2', 'lastUpdateTime': None, 'maxCombo': 0, 'name': 'f71cdd9b4b874da2ba7a186c31c380b2', 'parameters': {'batch': {'max': 100, 'min': 60, 'scalingType': 'uniform', 'type': 'integer'}, 'epoch': {'max': 20, 'min': 5, 'scalingType': 'uniform', 'type': 'integer'}}, 'predictor': None, 'spec': {'gridSize': 10, 'maxCombo': 0, 'metric': 'zzxx', 'minSampleSize': 100, 'objective': 'maximize', 'retryAssignLimit': 1, 'retryLimit': 20, 'seed': 419048492}, 'startTime': 20293427104, 'state': {'sequence_i': 0, 'sequence_pid': None, 'sequence_retry': 0}, 'status': 'running', 'suggestion_count': 0, 'trials': 1, 'version': '1.0.24'}


**Start experiments**  
https://www.comet.ml/docs/python-sdk/introduction-optimizer/

In [12]:
for experiment in opt.get_experiments():
    zzxx=train(x_train,y_train,x_test,y_test,batch=experiment.get_parameter("batch"),epoch=experiment.get_parameter("epoch"))
    experiment.log_metric("zzxx",zzxx)
    

COMET INFO: Experiment is live on comet.ml https://www.comet.ml/returncode13/optimizer-search-02/b471ee7163e24903b02cd34128c8cd3d



COMET INFO: Ignoring automatic log_parameter('verbose') because 'keras:verbose' is in COMET_LOGGING_PARAMETERS_IGNORE
COMET INFO: Ignoring automatic log_parameter('do_validation') because 'keras:do_validation' is in COMET_LOGGING_PARAMETERS_IGNORE


Train on 60000 samples, validate on 10000 samples
Epoch 1/12
   95/60000 [..............................] - ETA: 2:47 - loss: 2.4796 - accuracy: 0.1053

COMET INFO: Ignoring automatic log_metric('batch_batch') because 'keras:batch_batch' is in COMET_LOGGING_METRICS_IGNORE
COMET INFO: Ignoring automatic log_metric('batch_size') because 'keras:batch_size' is in COMET_LOGGING_METRICS_IGNORE


60000/60000 [==============================] - 3s 45us/step - loss: 0.6565 - accuracy: 0.8059 - val_loss: 0.2808 - val_accuracy: 0.9164
Epoch 2/12
60000/60000 [==============================] - 2s 40us/step - loss: 0.2380 - accuracy: 0.9290 - val_loss: 0.1970 - val_accuracy: 0.9399
Epoch 3/12
60000/60000 [==============================] - 2s 40us/step - loss: 0.1689 - accuracy: 0.9494 - val_loss: 0.1615 - val_accuracy: 0.9523
Epoch 4/12
60000/60000 [==============================] - 2s 40us/step - loss: 0.1334 - accuracy: 0.9602 - val_loss: 0.1404 - val_accuracy: 0.9590
Epoch 5/12
60000/60000 [==============================] - 3s 43us/step - loss: 0.1098 - accuracy: 0.9672 - val_loss: 0.1240 - val_accuracy: 0.9619
Epoch 6/12
60000/60000 [==============================] - 2s 40us/step - loss: 0.0930 - accuracy: 0.9715 - val_loss: 0.1104 - val_accuracy: 0.9672
Epoch 7/12
60000/60000 [==============================] - 2s 40us/step - loss: 0.0814 - accuracy: 0.9744 - val_loss: 0.1106 - val

...

60000/60000 [==============================] - 3s 51us/step - loss: 0.0389 - accuracy: 0.9883 - val_loss: 0.0788 - val_accuracy: 0.9784
Epoch 14/17
38850/60000 [==================>...........] - ETA: 0s - loss: 0.0340 - accuracy: 0.9899

51150/60000 [========================>.....] - ETA: 0s - loss: 0.0351 - accuracy: 0.9895

59250/60000 [============================>.] - ETA: 0s - loss: 0.0359 - accuracy: 0.9894

60000/60000 [==============================] - 3s 51us/step - loss: 0.0358 - accuracy: 0.9894 - val_loss: 0.0851 - val_accuracy: 0.9787
Epoch 15/17
60000/60000 [==============================] - 3s 50us/step - loss: 0.0318 - accuracy: 0.9901 - val_loss: 0.0777 - val_accuracy: 0.9796
Epoch 16/17
60000/60000 [==============================] - 3s 50us/step - loss: 0.0287 - accuracy: 0.9909 - val_loss: 0.0791 - val_accuracy: 0.9786
Epoch 17/17
10000/10000 [==============================] - 1s 53us/step


ReadTimeout: HTTPSConnectionPool(host='optimizer.comet.ml', port=443): Read timed out. (read timeout=10)

In [32]:
#train(x_train,y_train,x_test,y_test)

In [33]:
experiment.end()

TypeError: object of type 'NoneType' has no len()